In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from bs4 import BeautifulSoup



In [5]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [7]:
df= []
url = 'https://www.google.com/maps/search/tax+filing+firm+in+hawaii/@42.8947107,-129.9427085,2z/data=!3m1!4b1?entry=ttu'


In [13]:
def Scrape_phase1( url , df):
    driver = webdriver.Chrome()

    # Open the URL in the browser
    driver.get(url)
    driver.implicitly_wait(10)

    # Define a wait for the presence of at least one div with class "Nv2PK"
    wait = WebDriverWait(driver, 10)
    all_nv2pk_divs = []


    try:
        while True:
            # Collect all div elements with class "Nv2PK"
            nv2pk_divs = driver.find_elements(By.CSS_SELECTOR, 'div.Nv2PK')

            # Add the found divs to the list if there are new ones
            if len(nv2pk_divs) > len(all_nv2pk_divs):
                all_nv2pk_divs = nv2pk_divs  # Update the collected divs list
                print(f"Collected {len(all_nv2pk_divs)} divs")

                # Scroll to a specific element that may trigger more content loading
                last_div = nv2pk_divs[-1]
                driver.execute_script("arguments[0].scrollIntoView(true);", last_div)

                # Introduce a sleep to allow time for content to load (adjust as needed)
                time.sleep(3)  # Adjust the time (in seconds) as needed

                # Wait for the newly loaded divs (optional)
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.Nv2PK')))

                # Get the updated HTML content after scrolling and waiting
                html_content = driver.page_source

                # Create a BeautifulSoup object to parse the updated HTML content
                soup = BeautifulSoup(html_content, 'html.parser')
                #print(soup)

                # Find all div elements with class "Nv2PK" using BeautifulSoup
                all_nv2pk_divs_soup = soup.find_all('div', class_='Nv2PK')
                print(len(all_nv2pk_divs_soup))

                # You can now work with the BeautifulSoup-selected elements
                for i in range(len(all_nv2pk_divs_soup)):
                    entry = []
                    
                    item = all_nv2pk_divs_soup[i].find("a")
                    link_of_item = item["href"]

                    driver.get(link_of_item)

                    src = driver.page_source
                    soup = BeautifulSoup(src, 'html.parser')

                    name_of_company = soup.find("h1").get_text()
                    print(name_of_company)


                    entry.append(name_of_company)


                    phone_number = "//www.gstatic.com/images/icons/material/system_gm/2x/phone_gm_blue_24dp.png"
                    address = "//www.gstatic.com/images/icons/material/system_gm/2x/place_gm_blue_24dp.png"
                    website_link = "//www.gstatic.com/images/icons/material/system_gm/2x/public_gm_blue_24dp.png"
                    Email = "NaN"

                    #phone_number
                    phone_number_icon = soup.find(src = phone_number)

                    if(phone_number_icon == None):
                        phone_number = "NaN"
                    else:
                        level2 = phone_number_icon.find_parents(limit = 3)
                        phone_number =level2[2].get_text()


                    #address
                    address_icon = soup.find(src = address)
                    if(address_icon == None):
                        address = "NaN"
                    else:
                        level2 = address_icon.find_parents(limit = 3)
                        address =level2[2].get_text()


                    #website_link
                    website_link_icon = soup.find(src = website_link)
                    if(website_link_icon == None):
                        website_link = "NaN"
                    else:
                        level2 = website_link_icon.find_parents(limit = 3)
                        website_link =level2[2].get_text()






                    entry.append(phone_number)
                    entry.append(address)
                    entry.append(website_link)
                    entry.append(Email)

                    print(entry)
                    df.append(entry)
            else:
                print("length not increased")
                break



    except (StaleElementReferenceException, TimeoutException) as e:
        # Handle StaleElementReferenceException or TimeoutException
        print(f"An exception occurred: {e}")

    finally:
        driver.quit()


SyntaxError: invalid syntax (4230245043.py, line 49)

In [9]:
def Scrape_phase2(leads):
    df = leads
    for ind in df.index:
        website = df['Website Link'][ind]
        if(website!= "NaN"):
            final_link = "https://" + website + "/"
            driver = webdriver.Chrome()
            driver.get(final_link)
            h= driver.page_source
            soup=BeautifulSoup(h,'html.parser')
            mailto_links = soup.find_all('a', href=lambda href: href and href.startswith('mailto:'))
            print(len(mailto_links))
            for link in mailto_links:
                email = link['href'].split(':')[1]
                df['Email'][ind] = email
                
  

In [10]:
Name  = input("Enter your Name of fime: ") 
Input_link = input("Enter your Input_link: ") 
df = []
Scrape_phase1(Input_link, df)

leads=pd.DataFrame(df, columns=['Name of company','Phone Number',  'Location',  'Website Link', 'Email'])
Scrape_phase2(leads)
leads.head()
leads.to_excel(Name+".xlsx", index=True)
leads.to_csv(Name + ".csv", index= True)

Enter your Name of fime: tax
Enter your Input_link: https://www.google.com/maps/search/tax+filing+firm+in+hawaii/@42.8947107,-129.9427085,2z/data=!3m1!4b1?entry=ttu
Collected 8 divs
15
Smart Tax & Business Solutions, Inc.
['Smart Tax & Business Solutions, Inc.', '(808) 354-0208', '500 Ala Moana Blvd Building 7, Suite 400, Honolulu, HI 96813', 'smarttaxhawaii.com', 'NaN']
Tax Services of Hawaii Inc
['Tax Services of Hawaii Inc', '(808) 529-1040', '841 Bishop St #2105, Honolulu, HI 96813', 'taxservicesofhawaii.com', 'NaN']
Diamond Head Tax Group Individual and Business Tax Advisory Honolulu
['Diamond Head Tax Group Individual and Business Tax Advisory Honolulu', '(808) 824-3588', '1441 Kapiolani Blvd Suite 912, Honolulu, HI 96814', 'diamondheadtaxgroup.com', 'NaN']
H&R Block
['H&R Block', '(808) 487-0911', '4510 Salt Lake Blvd Ste A-8, Honolulu, HI 96818', 'hrblock.com', 'NaN']
E.A. Buck Accounting & Tax Services
['E.A. Buck Accounting & Tax Services', '(808) 395-5492', '55 Merchant St S

NameError: name 'Namw' is not defined

In [11]:
leads.to_csv(Name + ".csv", index= True)


In [12]:
final_frame = pd.read_csv("tax.csv")
final_frame

,Unnamed: 0,Name of company,Phone Number,Location,Website Link,Email
0,0,"Smart Tax & Business Solutions, Inc.",(808) 354-0208,"500 Ala Moana Blvd Building 7, Suite 400, Hono...",smarttaxhawaii.com,NaN
1,1,Tax Services of Hawaii Inc,(808) 529-1040,"841 Bishop St #2105, Honolulu, HI 96813",taxservicesofhawaii.com,support@myhawaiitax.com
2,2,Diamond Head Tax Group Individual and Business...,(808) 824-3588,"1441 Kapiolani Blvd Suite 912, Honolulu, HI 96814",diamondheadtaxgroup.com,NaN
3,3,H&R Block,(808) 487-0911,"4510 Salt Lake Blvd Ste A-8, Honolulu, HI 96818",hrblock.com,NaN
4,4,E.A. Buck Accounting & Tax Services,(808) 395-5492,"55 Merchant St STE 2100, Honolulu, HI 96813",eabucktaxes.com,NaN
5,5,Certified Tax Hawaii,(808) 842-0003,"1405 N King St # 103, Honolulu, HI 96817",NaN,NaN
6,6,"Jung Hoon Ji, CPA",(808) 909-3338,"1441 Kapiolani Blvd SUITE 716, Honolulu, HI 96814",junghoonjicpa.com,NaN
7,7,Accurate Affordable Accounting and Tax Service...,(808) 396-6507,"377 Keahole St # E203, Honolulu, HI 96825",lookintohawaii.com,NaN
8,8,Hawaii Tax Services LLC,(808) 388-4073,"650 Iwilei Rd Suite 201-B, Honolulu, HI 96817",business.site,NaN
9,9,Tax Relief Services,(808) 589-2322,"1259 S Beretania St. # 2, Honolulu, HI 96814",taxreliefservices.com,irstaxrelief@msn.com
